In [5]:
import numpy as np
import pandas as pd
import scipy.optimize
from scipy import stats

import powerlaw
import pickle5 as pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#Unpickling the data
def load_obj(name):
    with open('resources/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [3]:
# Bus adjacency
def bus_adjacency(stoproute,lsoa_list,route_freqs):
    
    # Create matrix that combines location data and route frequencies
    combine = pd.merge(stoproute, route_freqs, on='line')
    combine = combine.drop_duplicates(['line', 'naptan_sto'])
    combine = combine.rename(columns={'geo_code':'lsoa11cd'})

    # Create adjacency matrix LSOA x route
    bstopfreq = combine[['lsoa11cd', 'naptan_sto', 'line', 'average']]
    adj = pd.pivot(bstopfreq,index=["lsoa11cd", "naptan_sto"], columns="line", values="average").fillna(0)
    adj = adj.astype(float)
    adj = adj.groupby(level="lsoa11cd").mean()
    bus2route = pd.merge(lsoa_list, adj, how='left',on='lsoa11cd').set_index('lsoa11cd')

    #Adjacency matrix LSOA x LSOA
    bus2route = np.array(bus2route)
    bus2routeT = bus2route.transpose()
    lsoa2lsoa = np.dot(bus2route,bus2routeT)**0.5 #check that this actually does whay I think it does
    lsoa2lsoa[np.diag_indices_from(lsoa2lsoa)] = 0

    lsoa2lsoa = pd.DataFrame(lsoa2lsoa)
    lsoa2lsoa = lsoa2lsoa.fillna(0)

    #m values created
    m_bus = np.round(lsoa2lsoa.copy(),0)
    m_bus[m_bus>0]=np.log10(m_bus[m_bus>0])
    m_bus=1-(m_bus/np.max(np.max(m_bus)))
    m_bus[m_bus==0]=np.min(np.min(m_bus[m_bus!=0]))
    
    return m_bus.values

# Attractivity samples - median/neighbourhood, directional 
def attractivity_median_sampler(oa, edu_ratios, income_params, size):
   
    edu = np.random.choice(4, size = size, p=edu_ratios[oa]) #where p values are effectively the ratio of people with a given education level
    income = stats.beta.rvs(income_params[oa, 0], income_params[oa, 1], loc = income_params[oa, 2], scale = income_params[oa, 3], size=size)

    attractivity = np.power(income, -edu)

    return np.median(attractivity)



# Median att matrix 
def median_attractivity(edu_ratios, income_params): #,fit = None):

    """
    Average individual attractivity / lsoa (taken as a sample of 1000 ppl)
    Sample is directinal - matrix not symmetrical
    """

    attractivity = np.zeros((len(income_params)))
    size = 10000

    for i in range(len(income_params)):
        attractivity[i] = attractivity_median_sampler(i, edu_ratios, income_params, size)

    attractivity = attractivity.reshape((len(attractivity),1))

    return attractivity



In [6]:
# Input data 
lsoa_data = load_obj("newdata_lsoa_data")
sheff_shape, income_params, edu_counts, edu_ratios = lsoa_data['sheff_lsoa_shape'], lsoa_data['income_params'], lsoa_data['edu_counts'], lsoa_data['edu_ratios']

# k
comp_ratio = np.load("resources/newdata_companyhouse.npy")

# distances
paths_matrix = load_obj("newdata_ave_paths")
# removes all 0s not on the diag 
paths_matrix[paths_matrix==0] = 1
paths_matrix[np.diag_indices_from(paths_matrix)] = 0

# bus freq paths 
stoproute = pd.read_csv('resources/stoproute_withareacodes.csv')
lsoa_list = pd.read_csv("resources/E47000002_KS101EW.csv")['lsoa11cd']
route_freqs = pd.read_csv('resources/Bus_routes_frequency.csv', usecols= ["line","average"]).astype(str)
m_paths = bus_adjacency(stoproute, lsoa_list, route_freqs)


attractivity_avg = median_attractivity(edu_ratios, income_params)

#population amplification
pop = np.asarray(edu_counts).reshape((len(edu_counts), 1))

pop = np.matmul(pop, pop.transpose())

#connectivity matrix
attractivity_product = np.matmul(attractivity_avg, attractivity_avg.transpose())
attractivity_product = np.multiply(attractivity_product, comp_ratio)

#ensure 0 on diagonal?
connectivity = np.divide(attractivity_product, np.power(paths_matrix, m_paths))
connectivity[np.where(np.isinf(connectivity))[0], np.where(np.isinf(connectivity))[1]] = 0
connectivity[np.diag_indices_from(connectivity)] = 0

/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)
/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_56854/2810495905.py:33: RuntimeWarning: divide by zero encountered in true_divide
  connectivity = np.divide(attractivity_product, np.power(paths_matrix, m_paths))
/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_56854/2810495905.py:33: Runt

# Test run for a 10 x 10 matrix 

In [53]:
attractivity_p = attractivity_product[0:10,0:10]
paths_mat = paths_matrix[0:10,0:10]
m_pat = m_paths[0:10,0:10]
np.sum(np.sum(m_pat))

82.2082223174257

In [58]:
from scipy.optimize import minimize

# objective function - sum of connectivity
def m_opt(m):
    
    # attractivity product
    lsoa_data = load_obj("newdata_lsoa_data")
    sheff_shape, income_params, edu_counts, edu_ratios = lsoa_data['sheff_lsoa_shape'], lsoa_data['income_params'], lsoa_data['edu_counts'], lsoa_data['edu_ratios']
    comp_ratio = np.load("resources/newdata_companyhouse.npy")
    attractivity_avg = median_attractivity(edu_ratios, income_params)
    attractivity_product = np.matmul(attractivity_avg, attractivity_avg.transpose())
    attractivity_product = np.multiply(attractivity_product, comp_ratio)
    
    # distances
    paths_matrix = load_obj("newdata_ave_paths")
    # removes all 0s not on the diag 
    paths_matrix[paths_matrix==0] = 1
    paths_matrix[np.diag_indices_from(paths_matrix)] = 0
    
    # bus network
    stoproute = pd.read_csv('resources/stoproute_withareacodes.csv')
    lsoa_list = pd.read_csv("resources/E47000002_KS101EW.csv")['lsoa11cd']
    route_freqs = pd.read_csv('resources/Bus_routes_frequency.csv', usecols= ["line","average"]).astype(str)
    m_paths = bus_adjacency(stoproute, lsoa_list, route_freqs)
    
    attractivity_p = attractivity_product[0:10,0:10]
    paths_mat = paths_matrix[0:10,0:10]
    m_pat = m_paths[0:10,0:10]
    
    m = np.reshape(m,(len(m_pat),len(m_pat)))
    f = np.divide(attractivity_p, np.power(paths_mat, m)) # needs removing of inf/nan
    f[np.where(np.isinf(f))[0], np.where(np.isinf(f))[1]] = 0
    f[np.diag_indices_from(f)] = 0
    f1 = - np.sum(np.sum(f))
    
    return f1


# calculating the gradient of above function 
def opt_der(m):
    
    m = np.reshape(m,(len(m_pat),len(m_pat)))
    der = np.zeros_like(m) # don't think it's needed 
    der = np.multiply(attractivity_p, np.power(paths_mat, -m), -m)
    der = sum(sum(np.multiply(np.log(attractivity_p),der)))
    
    return der

In [54]:
ineq_cons = {'type': 'ineq',
              'fun' : lambda m: np.array([(1.05 * 82) - np.sum(np.sum(m)),   #A1*sum M - sum m
                                          np.sum(np.sum(m)) - (0.95 * 82)])} #sum m - A2*sum M
#               'jac' : lambda x: np.array([[-1.0],
#                                           [1.0]])}
eq_cons = {'type': 'eq',
            'fun' : lambda m: np.array([np.median(m_pat) - np.median(m)])}
            #'jac' : lambda x: np.array(0)}  #??
    
# if eq_cons is used, the line gives the following error (bc constraint is always = 0?): Singular matrix C in LSQ subproblem    (Exit mode 6) after 1 iteration 
# if only ineq_cons used, gives the following error: Singular matrix E in LSQ subproblem    (Exit mode 5)
# if only use ineq_cons and start with an x0 = np.ones_like(m_pat): Inequality constraints incompatible    (Exit mode 4)

In [59]:
x0 = np.ones_like(m_pat)
# x0 = m_pat.copy()
res = minimize(m_opt, x0, method='SLSQP', #jac=opt_der,
            constraints=[ineq_cons], options={'disp': True})
            #bounds=bounds)
# may vary

print(np.reshape(res.x,(10,10)))

/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)
/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_56854/1272124461.py:31: RuntimeWarning: divide by zero encountered in true_divide
  f = np.divide(attractivity_p, np.power(paths_mat, m)) # needs removing of inf/nan
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core

/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)
/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_56854/1272124461.py:31: RuntimeWarning: divide by zero encountered in true_divide
  f = np.divide(attractivity_p, np.power(paths_mat, m)) # needs removing of inf/nan
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core

/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)
/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_56854/1272124461.py:31: RuntimeWarning: divide by zero encountered in true_divide
  f = np.divide(attractivity_p, np.power(paths_mat, m)) # needs removing of inf/nan
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core

/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)
/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_56854/1272124461.py:31: RuntimeWarning: divide by zero encountered in true_divide
  f = np.divide(attractivity_p, np.power(paths_mat, m)) # needs removing of inf/nan
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core

/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)
/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_56854/1272124461.py:31: RuntimeWarning: divide by zero encountered in true_divide
  f = np.divide(attractivity_p, np.power(paths_mat, m)) # needs removing of inf/nan
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core

...

/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)
/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_56854/1272124461.py:31: RuntimeWarning: divide by zero encountered in power
  f = np.divide(attractivity_p, np.power(paths_mat, m)) # needs removing of inf/nan
/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_56854/1272124461.py:31: RuntimeW

/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)
/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_56854/1272124461.py:31: RuntimeWarning: divide by zero encountered in power
  f = np.divide(attractivity_p, np.power(paths_mat, m)) # needs removing of inf/nan
/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_56854/1272124461.py:31: RuntimeW

/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)
/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_56854/1272124461.py:31: RuntimeWarning: divide by zero encountered in power
  f = np.divide(attractivity_p, np.power(paths_mat, m)) # needs removing of inf/nan
/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_56854/1272124461.py:31: RuntimeW

Singular matrix E in LSQ subproblem    (Exit mode 5)
            Current function value: -3.84279798687754e+272
            Iterations: 2
            Function evaluations: 210
            Gradient evaluations: 2
[[ 2.62334041e+03  4.90481185e+02  7.73897352e+03 -2.53940530e+03
   3.54016371e+03  6.55714503e+02 -1.97809045e+04 -5.66780514e+03
  -6.91654971e+03 -1.47966759e+03]
 [ 1.00904613e+04 -1.42410971e+04  6.79841940e+03 -1.59307453e+03
  -3.60567794e+03  1.53840854e+04  6.62776117e+03  6.55557472e+03
  -8.23654467e+02  6.08382257e+03]
 [ 1.41964555e+04  1.45389584e+03  1.32661783e+04 -6.07615787e+03
   6.37250486e+03  1.18251989e+03  2.70696910e+03 -2.03007824e+02
  -2.39458957e+03  5.67773120e+03]
 [-1.44189356e+04 -2.26051777e+03  8.17075298e+01 -1.93371928e+03
  -1.24773124e+03  1.86868902e+04  8.86205794e+03  7.61163656e+03
  -4.93703996e+02  1.64158698e+03]
 [ 1.37181758e+03 -1.27161137e+03  2.11415518e+03 -2.00872331e+04
  -2.53822414e+03  1.65693661e+03  6.54339407e+03  4.4

/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)
/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_56854/1272124461.py:31: RuntimeWarning: divide by zero encountered in power
  f = np.divide(attractivity_p, np.power(paths_mat, m)) # needs removing of inf/nan
/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_56854/1272124461.py:31: RuntimeW

# Trying the 'trust-constr' method

In [90]:
from scipy.optimize import Bounds
[[0]]
bounds = Bounds([0 for i in range(len(m_pat)**2)], [1.0 for i in range(len(m_pat)**2)])
bounds

Bounds(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]))

In [89]:
from scipy.optimize import NonlinearConstraint

def cons_f(m):
    g1 = (1.05 * 82) - np.sum(np.sum(m))
    g2 = np.sum(np.sum(m)) - (0.95 * 82)
    return [g1, g2]

nonlinear_constraint = NonlinearConstraint(cons_f, 0, np.inf)

x0 = np.ravel(np.ones_like(m_pat))
res = minimize(m_opt, x0, method='trust-constr', #jac=rosen_der, hess=rosen_hess,
               constraints=[nonlinear_constraint],
               options={'verbose': 1}, 
               bounds=bounds)
    
print(res.x)

/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)
/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_56854/1272124461.py:31: RuntimeWarning: divide by zero encountered in true_divide
  f = np.divide(attractivity_p, np.power(paths_mat, m)) # needs removing of inf/nan
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core

/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)
/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_56854/1272124461.py:31: RuntimeWarning: divide by zero encountered in true_divide
  f = np.divide(attractivity_p, np.power(paths_mat, m)) # needs removing of inf/nan
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core

/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)
/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_56854/1272124461.py:31: RuntimeWarning: divide by zero encountered in true_divide
  f = np.divide(attractivity_p, np.power(paths_mat, m)) # needs removing of inf/nan
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core

/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)
/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_56854/1272124461.py:31: RuntimeWarning: divide by zero encountered in true_divide
  f = np.divide(attractivity_p, np.power(paths_mat, m)) # needs removing of inf/nan
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core

/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)
/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_56854/1272124461.py:31: RuntimeWarning: divide by zero encountered in true_divide
  f = np.divide(attractivity_p, np.power(paths_mat, m)) # needs removing of inf/nan
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core

...

/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)
/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_56854/1272124461.py:31: RuntimeWarning: divide by zero encountered in true_divide
  f = np.divide(attractivity_p, np.power(paths_mat, m)) # needs removing of inf/nan
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core

/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)
/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_56854/1272124461.py:31: RuntimeWarning: divide by zero encountered in true_divide
  f = np.divide(attractivity_p, np.power(paths_mat, m)) # needs removing of inf/nan
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core

/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)
/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_56854/1272124461.py:31: RuntimeWarning: divide by zero encountered in true_divide
  f = np.divide(attractivity_p, np.power(paths_mat, m)) # needs removing of inf/nan
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core

/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)
/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_56854/1272124461.py:31: RuntimeWarning: divide by zero encountered in true_divide
  f = np.divide(attractivity_p, np.power(paths_mat, m)) # needs removing of inf/nan


KeyboardInterrupt: 

In [85]:
#x0 = np.reshape(np.ones_like(m_pat),(1,100))

x0 = np.array(np.random.rand(10,10))

x0 = np.ravel(x0)

x0

array([0.42409691, 0.93767937, 0.78686781, 0.0497983 , 0.02974155,
       0.55451091, 0.65144953, 0.5565349 , 0.97476344, 0.08123866,
       0.61026532, 0.48470936, 0.30739805, 0.12105577, 0.41538816,
       0.73989451, 0.22818617, 0.66395938, 0.05019345, 0.32155446,
       0.65397337, 0.39811818, 0.86145721, 0.66310163, 0.23448853,
       0.06409941, 0.93856458, 0.28413806, 0.74143151, 0.82940362,
       0.5515717 , 0.17171412, 0.91687792, 0.73197078, 0.82424402,
       0.00278279, 0.74123034, 0.42805776, 0.63536797, 0.5587373 ,
       0.82626137, 0.11417358, 0.83496738, 0.10171991, 0.78278471,
       0.52546676, 0.36862543, 0.942444  , 0.30803163, 0.84573513,
       0.45476792, 0.51183665, 0.69466485, 0.2658089 , 0.68244733,
       0.55681024, 0.81672094, 0.8920237 , 0.64542823, 0.52383825,
       0.49334914, 0.63086034, 0.55257067, 0.29712225, 0.34248851,
       0.75068582, 0.81248465, 0.82277817, 0.98062298, 0.61412536,
       0.5893936 , 0.50288354, 0.42373885, 0.47101225, 0.38107